<a href="https://colab.research.google.com/github/AlexWortega/Collab_experement/blob/main/Binanance_bot_trade_mt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-binance
!pip install websocket-client

In [ ]:
import json
import pprint
import numpy
import websocket
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
import numpy as np
import pickle
from binance.client import Client

In [ ]:
client = Client('pb_key', 'sc_key')
balance = client.get_asset_balance(asset='BNB')

In [ ]:
balance["free"]

In [ ]:
import os
from binance.client import Client
import pandas as pd
import datetime, time

def GetHistoricalData( howLong,pair='BNBUSDT'):
    howLong = howLong
    # Calculate the timestamps for the binance api function
    untilThisDate = datetime.datetime.now()
    sinceThisDate = untilThisDate - datetime.timedelta(days = howLong)

   
   
    # Execute the query from binance - timestamps must be converted to strings !


    candle = client.get_historical_klines(pair, Client.KLINE_INTERVAL_1MINUTE, str(sinceThisDate), str(untilThisDate))

    # Create a dataframe to label all the columns returned by binance so we work with them later.
    
    
    df = pd.DataFrame(candle, columns=['dateTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'])

    df.dateTime = pd.to_datetime(df.dateTime, unit='ms').dt.strftime(Constants.DateTimeFormat)
    df.set_index('dateTime', inplace=True)

    
    df = df.drop(['closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol','takerBuyQuoteVol', 'ignore'], axis=1)
    return df


In [ ]:
df = GetHistoricalData(1000)

In [ ]:
df.info()

In [ ]:
df.to_csv('BNBUSDT.csv')

# Train

In [ ]:
data = pd.read_csv('BNBUSDT.csv', skiprows=[0])

In [ ]:
df = data.drop(["date", "symbol","Volume BTC", "Volume USDT", "tradecount"], axis=1)

In [ ]:
hour_data = pd.DataFrame(columns=[*range(1,121)])
hour_data["end"] = 0
for i in range(1,10000):
  values = list(df['open'][60+30*i:180+30*i])
  values.append(df['close'][30*i])
  hour_data.loc[len(hour_data)] = values

In [ ]:
hour_data.head()

In [ ]:
X = hour_data.drop('end', axis=1)
y = hour_data['end']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
reg1 = GradientBoostingRegressor(n_estimators=100)
reg2 = RandomForestRegressor()
reg3 = LinearRegression()
reg4 = linear_model.Ridge(alpha=.5)

ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3),('ri',reg4)])

ereg.fit(X_train, y_train)

In [ ]:
ereg

# Test

In [ ]:
 
pkl_filename = "binance_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(ereg, file)

In [ ]:
closes = []
time = []
have = []
expired = []

In [ ]:
reg1 = GradientBoostingRegressor(n_estimators=100)
reg2 = RandomForestRegressor()
reg3 = LinearRegression()
reg4 = linear_model.Ridge(alpha=.5)

ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3),('ri',reg4)])


In [ ]:
ereg = pickle.load(open('/content/binance_model.pkl', 'rb'))

In [ ]:
SOCKET = "wss://stream.binance.com:9443/ws/btcusdt@kline_1m"




def on_open(ws):
    print('opened connection')


def on_close(ws):
    print('closed connection')

# every time btc value closes
def on_message(ws, message):
    global closes, i    

    json_message = json.loads(message)

    candle = json_message['k']
    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        closes.append(float(close))
        #if more than 2 hours passed after we started program

        if len(closes)>120:
            last_hours = np.array(closes[-120:]) 
            #predict value of last 2 hours and compare with the last one

            if closes[-1]<float(loaded_model.predict(last_hours.reshape(1,-1))):
                #buy btc if pred val > last val 
                balance_usd = client.get_asset_balance(asset='USDT')
                if float(balance_usd['free'])>16:
                    order = client.create_order(
                        symbol='BNBUSDT',
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity = 0.0003)   
                    #add bought time and at what price we bought

                    time.append(int(candle['t'])+3600000)
                    print("bought for {}".format(float(closes[-1])))
                    have.append(float(closes[-1]))
        
        #if bought time was more than 2 hours ago than add bought val to expired    

        expired = have[:sum(i<candle['t'] for i in time)]
        if len(expired)>0:
            
            #if val in expired < than value now then sell

            if min(expired)<closes[-1]:                
                    balance_btc = client.get_asset_balance(asset='BTC')
                    if float(balance_btc['free'])>0.0003:
                        print('sold for {}'.format(closes[-1]))
                        order = client.create_order(
                            symbol='BTCUSDT',
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity = 0.0003) 
                        #del sold value
                        del time[have.index(min(expired))]
                        have.remove(min(expired))                    
                        expired.remove(min(expired))
                    
                    


ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()